In [1]:

#!python3 -m pip install pytorch_lightning
#!pip3 install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu113
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from model import LSTM_Model
from pytorch_lightning import seed_everything, LightningModule, Trainer
from torch import save
from pytorch_lightning.callbacks import EarlyStopping
from torch.utils.data import DataLoader
import neptune.new as neptune
from pytorch_lightning.loggers import NeptuneLogger



torch.manual_seed(1)
#!python3 -m pip install neptune-client

# Install pip packages in the current Jupyter kernel
import sys
#!{sys.executable} -m pip install torchinfo
#!{sys.executable} -m pip install neptune-notebooks
#!{sys.executable} -m pip install neptune-client

#!pip install neptune-notebooks
#!jupyter nbextension enable --py neptune-notebooks


Dataset version: 2.0


In [ ]:
seed_everything(42)
device = 'cuda' if torch.cuda.is_available() else 'cpu' #Check for cuda 
print(f'Using {device} device')

neptune_logger = NeptuneLogger(
    project="NTLAB/BCM-activity-classification",
    api_token="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiIxYTA4NzcxMy1lYmQ2LTQ3NTctYjRhNC02Mzk1NjdjMWM0NmYifQ==",
    source_files=["train_model.ipynb", "model.py", "BCM_dataset_v2.py"]
) 

model = LSTM_Model('data/bcm/', window_size = 3, stride = 0.032,lstm_hidden_size = 128,occlusion = [1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0])

trainer = Trainer(max_epochs=200, min_epochs=1, auto_lr_find=False, auto_scale_batch_size=False,enable_checkpointing=False, accelerator="gpu", devices = 1, logger=neptune_logger)
trainer.tune(model)
trainer.fit(model)


Global seed set to 42


Using cuda device
Validation set
data/bcm//validation/0.npy
data/bcm//validation/1.npy
data/bcm//validation/2.npy
data/bcm//validation/3.npy
data/bcm//validation/4.npy
Training set
data/bcm//train/0.npy
data/bcm//train/1.npy
data/bcm//train/2.npy
data/bcm//train/3.npy
data/bcm//train/4.npy


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


https://app.neptune.ai/NTLAB/BCM-activity-classification/e/BCMAC-26
Remember to stop your run once you’ve finished logging your metadata (https://docs.neptune.ai/api-reference/run#.stop). It will be stopped automatically only when the notebook kernel/interactive console is terminated.



  | Name         | Type             | Params
--------------------------------------------------
0 | lstm         | LSTM             | 149 K 
1 | flatten      | Flatten          | 0     
2 | fc           | Linear           | 1.3 K 
3 | output       | Sigmoid          | 0     
4 | sm           | Softmax          | 0     
5 | loss         | CrossEntropyLoss | 0     
6 | accuracy     | Accuracy         | 0     
7 | val_accuracy | Accuracy         | 0     
--------------------------------------------------
150 K     Trainable params
0         Non-trainable params
150 K     Total params
0.603     Total estimated model params size (MB)


...

{'progress_bar': {'val_loss': 1.4264299869537354}, 'log': {'val_loss': 1.4264299869537354}, 'val_loss': 1.4264299869537354}
Accuracy: 0.9361476898193359
Accuracy: 0.9990967512130737


Validation: 0it [00:00, ?it/s]

{'progress_bar': {'val_loss': 1.423315405845642}, 'log': {'val_loss': 1.423315405845642}, 'val_loss': 1.423315405845642}
Accuracy: 0.9476920366287231
Accuracy: 0.9988947510719299


Validation: 0it [00:00, ?it/s]

{'progress_bar': {'val_loss': 1.4236186742782593}, 'log': {'val_loss': 1.4236186742782593}, 'val_loss': 1.4236186742782593}
Accuracy: 0.9473693370819092
Accuracy: 0.9986748695373535


In [ ]:
seed_everything(42)
device = 'cuda' if torch.cuda.is_available() else 'cpu' #Check for cuda 
print(f'Using {device} device')

neptune_logger2 = NeptuneLogger(
    project="NTLAB/BCM-activity-classification",
    api_token="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiIxYTA4NzcxMy1lYmQ2LTQ3NTctYjRhNC02Mzk1NjdjMWM0NmYifQ==",
    source_files=["train_model.ipynb", "model.py", "BCM_dataset_v2.py"]
) 

model = LSTM_Model('data/bcm/', window_size = 3, stride = 0.032,lstm_hidden_size = 128,occlusion = [0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1])

trainer = Trainer(max_epochs=200, min_epochs=1, auto_lr_find=False, auto_scale_batch_size=False,enable_checkpointing=False, accelerator="gpu", devices = 1, logger=neptune_logger2)
trainer.tune(model)
trainer.fit(model)

In [ ]:
#torch.save(model.state_dict(), "trained_models/oct_6_3_sec_window")

model_scripted = torch.jit.script(model) # Export to TorchScript
#model_scripted.save("trained_models/oct_14") # Save